In [1]:
import math

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pandas as pd
import pyspark
import numpy as np
from pyspark import SparkConf, SparkContext
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from datetime import datetime
import pyspark.sql.functions as func
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=SparkConf()).getOrCreate()
device = 0
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
print(device)

cuda:0


In [2]:
# 定义常量
bs, T, i_size, h_size = 2, 3, 4, 5  #batch_size  Time  input_size hidden_size
proj_size = 3  #投影大小
input = torch.randn(bs, T, i_size)
c0 = torch.randn(bs, h_size)  # 初始值 不参与训练
h0 = torch.randn(bs, proj_size)  # 初始值 不参与训练

# 调用官方API
lstm_layer = nn.LSTM(i_size, h_size, batch_first=True, proj_size=proj_size)
output, (h_n, c_n) = lstm_layer(input, (h0.unsqueeze(0), c0.unsqueeze(0)))
print(output)

for k, v in lstm_layer.named_parameters():
    print(k, v.shape)

tensor([[[-0.2121,  0.2959,  0.0945],
         [-0.1147,  0.1519, -0.0013],
         [-0.1723,  0.2159, -0.0455]],

        [[ 0.0334, -0.0326,  0.2100],
         [-0.0067,  0.0364,  0.0578],
         [ 0.0970, -0.0321, -0.0943]]], grad_fn=<TransposeBackward0>)
weight_ih_l0 torch.Size([20, 4])
weight_hh_l0 torch.Size([20, 3])
bias_ih_l0 torch.Size([20])
bias_hh_l0 torch.Size([20])
weight_hr_l0 torch.Size([3, 5])


D:\zzf\python\pytorch\venv\Lib\site-packages\torch\nn\modules\rnn.py:879: UserWarning: LSTM with projections is not supported with oneDNN. Using default implementation. (Triggered internally at ..\aten\src\ATen\native\RNN.cpp:1493.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


In [4]:
import time

# 导入数据
# rawdata = spark.read.csv('data\\RAIN_H.csv')
rawdata = pd.read_csv('data\\QYMYL_LS.csv')
# rawdata.columns = ['BillingDate', 'VolumnHL']
print(rawdata)
# rawdata = rawdata.drop(columns='dt')
# rawdata = rawdata.drop(columns='ty')
# rawdata = rawdata.drop(columns='st')
# rawdata = rawdata.drop(columns='le')
# rawdata = rawdata.drop(columns='mt')
#
# time1 = time.time()
# df = pd.DataFrame(columns=['cz', 'pp', 'tm'])
# cz = []
# # for k, v in rawdata.iteritems():
# #     if (k == 'cz'):
# #         cz = v
# for k, v in rawdata.iterrows():
#     if v['cz'] == '624H4020':
#         df = df.append(v, ignore_index=True)
#     # print(v['cz'])
#
# df.to_csv('data/groupByCZ/test.csv')
# print(df)
# all_cz = ['624H4020', '70107662', '63300200', '63300800', '63005000', '70700127', '70400050', '70508428', '70700520',
#           '7011DA28', '63003200', '70700100', '70700144', '62414650', '70206220', '7011DE14', '70100500', '70107900',
#           '70400322', '63001000', '63002400', '63306325', '70101210', '70109151', '7011JH00', '70500600', '70124200',
#           '70205170', '70210000', '70503700', '70105450', '7012BH02', '62414710', '63310600', '7011DC00', '63005290',
#           '70509050', '63307700', '70100700', '70103200', '70403725', '63000100', '70109895', '70205085']
#
# time2 = time.time()
# print(time2 - time1)
# # print(rawdata)

FileNotFoundError: [Errno 2] No such file or directory: 'data\\QYMYL_LS.csv'

In [ ]:
import time

# 导入数据
# rawdata = spark.read.csv('data\\RAIN_H.csv')
rawdata = pd.read_csv('data\\QYMYL_LS.csv', low_memory=False)
# rawdata.columns = ['BillingDate', 'VolumnHL']
print(rawdata)
# rawdata.columns = ['RCD', 'TM', 'INTV', 'DRP', 'MODITIME', 'SOURCE']
# rawdata = rawdata.drop(columns='INTV')
# rawdata = rawdata.drop(columns='MODITIME')
# rawdata = rawdata.drop(columns='SOURCE')
#
print(rawdata)
time1 = time.time()
df = pd.DataFrame(columns=['RCD', 'TM', 'DRP'])
cz = []
# for k, v in rawdata.iteritems():
#     if (k == 'cz'):
#         cz = v
for k, v in rawdata.iterrows():
    if v['RCD'] == str:
        df = df.append(v, ignore_index=True)

print(df)
df.to_csv(f'data/groupByCZ/{str}MYL.csv')
all_cz = ['624H4020', '70107662', '63300200', '63300800', '63005000', '70700127', '70400050', '70508428', '70700520',
          '7011DA28', '63003200', '70700100', '70700144', '62414650', '70206220', '7011DE14', '70100500', '70107900',
          '70400322', '63001000', '63002400', '63306325', '70101210', '70109151', '7011JH00', '70500600', '70124200',
          '70205170', '70210000', '70503700', '70105450', '7012BH02', '62414710', '63310600', '7011DC00', '63005290',
          '70509050', '63307700', '70100700', '70103200', '70403725', '63000100', '70109895', '70205085']

time2 = time.time()
print(time2 - time1)
# print(rawdata)

In [9]:
# rawdata = pd.read_csv(f'data/groupByCZ/{str}MYL.csv', low_memory=False)
rawdata = df
count = 0
df1 = pd.DataFrame(columns=['RCD', 'TM', 'DRP'])
for k, v in rawdata.iterrows():
    if v['TM'] > '2021-01-01':
        # df1.append(v)
        print(v['TM'], v['DRP'])
        rawdata = rawdata.drop(k)
        count += 1

# rawdata = rawdata.drop([0, rawdata.shape[0]-365])
print(rawdata)
print(count)
rawdata.to_csv(f'data/groupByCZ/{str}MYL.csv')

2021-01-01 00:00:00 15.5
2021-01-02 00:00:00 8.5
2021-01-03 00:00:00 3.3
2021-01-04 00:00:00 0.5
2021-01-05 00:00:00 116.2
2021-01-06 00:00:00 100.3
2021-01-07 00:00:00 15.7
2021-01-08 00:00:00 7.7
2021-01-09 00:00:00 0.7
2021-01-10 00:00:00 1.7
2021-01-11 00:00:00 0.2
2021-01-12 00:00:00 0.2
2021-01-13 00:00:00 0.0
2021-01-14 00:00:00 0.3
2021-01-15 00:00:00 0.0
2021-01-16 00:00:00 0.0
2021-01-17 00:00:00 0.3
2021-01-18 00:00:00 0.0
2021-01-19 00:00:00 0.0
2021-01-20 00:00:00 0.0
2021-01-21 00:00:00 0.5
2021-01-22 00:00:00 908.1
2021-01-23 00:00:00 401.1
2021-01-24 00:00:00 5.7
2021-01-25 00:00:00 159.4
2021-01-26 00:00:00 296.4
2021-01-27 00:00:00 599.3
2021-01-28 00:00:00 34.0
2021-01-29 00:00:00 2.2
2021-01-30 00:00:00 0.2
2021-01-31 00:00:00 2.3
2021-02-01 00:00:00 688.9
2021-02-02 00:00:00 8.8
2021-02-03 00:00:00 2.0
2021-02-04 00:00:00 35.5
2021-02-05 00:00:00 403.7
2021-02-06 00:00:00 7.0
2021-02-07 00:00:00 12.2
2021-02-08 00:00:00 1288.5
2021-02-09 00:00:00 33.5
2021-02-10 00

In [7]:
df.to_csv(f'data/groupByCZ/{str}MYL.csv')

In [2]:
rawdata = spark.read.csv('data/groupByCZ/JiChang_2005_.csv')
rawdata.show()
# data = rawdata.filter(rawdata['_c0'] == '63000600')
# data.show()
pddata = rawdata.toPandas()
print(pddata)

+----+----------------+----+-----+-----+---+---+---+----+----+
| _c0|             _c1| _c2|  _c3|  _c4|_c5|_c6|_c7| _c8| _c9|
+----+----------------+----+-----+-----+---+---+---+----+----+
|null|            Time| Tem|  Pre| Pre1|Hum|Win|Vis| dew|Rain|
|   0|21.02.2023 14:00|10.1|771.4|775.4| 42|  2| 22|-2.2|   0|
|   1|21.02.2023 11:00| 8.3|773.5|777.5| 48|  3| 19|-2.1|   0|
|   2|21.02.2023 08:00|   6|773.1|777.1| 67|  4| 13| 0.3|   0|
|   3|21.02.2023 05:00| 5.4|771.8|775.9| 75|  2|  5| 1.3|   0|
|   4|21.02.2023 02:00| 6.5|771.4|775.4| 60|  3|  9|-0.7|   0|
|   5|20.02.2023 23:00| 7.8|770.4|774.4| 45|  2| 16|-3.4|   0|
|   6|20.02.2023 20:00| 9.8|769.3|773.3| 42|  2| 10|-2.5|   0|
|   7|20.02.2023 17:00|12.2|  768|  772| 31|  2| 13|-4.4|   0|
|   8|20.02.2023 14:00|  13|768.1|772.1| 28|  3| 15|-5.1|   0|
|   9|20.02.2023 11:00|10.5|769.3|773.4| 34|  3| 14|-4.7|   0|
|  10|20.02.2023 08:00| 6.8|768.7|772.8| 68|  2|  5| 1.3|   0|
|  11|20.02.2023 05:00| 5.6|767.2|771.2| 72|  1|  6| 0.

In [2]:
rawdata = pd.read_csv('data/groupByCZ/JiChang_2005_.csv')
# rawdata = rawdata.drop(columns='Unnamed: 0')
# rawdata = rawdata.iloc[::-1]
rawdata.Vis.astype('float')
rawdata.Pre.astype('float')
rawdata.Tem.astype('float')
rawdata.Dew.astype('float')
rawdata.Hum.astype('float')
rawdata.Pre1.astype('float')
rawdata.Rain.astype('float')
rawdata.Win.astype('float')
print(rawdata)


                   Time   Tem    Pre   Pre1  Hum  Win   Vis   Dew  Rain
0      01.02.2005 08:00   1.6  766.0  770.0   50    2  11.0  -7.7   0.0
1      01.02.2005 14:00   5.8  764.8  768.8   29    3  14.0 -10.9   0.0
2      01.02.2005 20:00   0.9  766.6  770.6   52    1  12.0  -7.8   0.0
3      02.02.2005 02:00  -0.7  767.0  771.1   62    1  10.0  -7.1   0.0
4      02.02.2005 08:00   0.2  767.2  771.3   63    1   4.0  -6.0   0.0
...                 ...   ...    ...    ...  ...  ...   ...   ...   ...
41198  21.02.2023 02:00   6.5  771.4  775.4   60    3   9.0  -0.7   0.0
41199  21.02.2023 05:00   5.4  771.8  775.9   75    2   5.0   1.3   0.0
41200  21.02.2023 08:00   6.0  773.1  777.1   67    4  13.0   0.3   0.0
41201  21.02.2023 11:00   8.3  773.5  777.5   48    3  19.0  -2.1   0.0
41202  21.02.2023 14:00  10.1  771.4  775.4   42    2  22.0  -2.2   0.0

[41203 rows x 9 columns]


In [16]:
import math


def checknum(x):
    if isinstance(x, float): return x
    y = x[0]
    if not y.isdigit():
        return 0
    else:
        return x


# for k, v in rawdata.iterrows():
#     print(k, v['Vis'], checknum(v['Vis']))
#     rawdata.loc[k, 'Vis'] = checknum(v['Vis'])

rawdata.fillna(method='ffill', inplace=True)
print(rawdata)

                   Time   Tem    Pre   Pre1   Hum  Win Vis   Dew  Rain
41202  01.02.2005 08:00   1.6  766.0  770.0  50.0  2.0  11  -7.7   0.0
41201  01.02.2005 14:00   5.8  764.8  768.8  29.0  3.0  14 -10.9   0.0
41200  01.02.2005 20:00   0.9  766.6  770.6  52.0  1.0  12  -7.8   0.0
41199  02.02.2005 02:00  -0.7  767.0  771.1  62.0  1.0  10  -7.1   0.0
41198  02.02.2005 08:00   0.2  767.2  771.3  63.0  1.0   4  -6.0   0.0
...                 ...   ...    ...    ...   ...  ...  ..   ...   ...
4      21.02.2023 02:00   6.5  771.4  775.4  60.0  3.0   9  -0.7   0.0
3      21.02.2023 05:00   5.4  771.8  775.9  75.0  2.0   5   1.3   0.0
2      21.02.2023 08:00   6.0  773.1  777.1  67.0  4.0  13   0.3   0.0
1      21.02.2023 11:00   8.3  773.5  777.5  48.0  3.0  19  -2.1   0.0
0      21.02.2023 14:00  10.1  771.4  775.4  42.0  2.0  22  -2.2   0.0

[41203 rows x 9 columns]


In [17]:
rawdata.to_csv(f'data/groupByCZ/JiChang_2005_1.csv')

In [7]:
def checkRun(year):
    if year % 400 == 0:
        return True
    if year % 4 == 0 and year % 100 != 0:
        return True
    return False


months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
print(sum(months))

df1 = pd.DataFrame(
    columns=['Time', 'minTem', 'maxTem', 'avgTem', 'avgPre', 'avgPre1', 'minHum', 'maxHum', 'avgHum', 'minWin',
             'maxWin',
             'avgWin', 'avgVis', 'avgDew',
             'avgRain'])
# def check(yy, mm, dd):


preday = '2005/2/1'
cnt = 0
minTem, maxTem = 99999.0, -99999.0
minHum, maxHum = 99999.0, -99999.0
minWin, maxWin = 99999.0, -99999.0
sumTem, sumPre, sumPre1, sumHum, sumWin, sumVis, sumDew, sumRain = 0, 0, 0, 0, 0, 0, 0, 0
for index, value in rawdata.iterrows():
    yy, mm, dd = int(value['Time'][6:10]), int(value['Time'][3:5]), int(value['Time'][0:2])
    today = f'{yy}/{mm}/{dd}'
    minTem = min(minTem, value['Tem'])
    maxTem = max(maxTem, value['Tem'])
    sumTem = sumTem + value['Tem']
    sumPre = sumPre + value['Pre']
    sumPre1 = sumPre1 + value['Pre1']
    minHum = min(minHum, value['Hum'])
    maxHum = max(maxHum, value['Hum'])
    sumHum = sumHum + value['Hum']
    minWin = min(minWin, value['Win'])
    maxWin = max(maxWin, value['Win'])
    sumWin = sumWin + value['Win']
    sumVis = sumVis + value['Vis']
    sumDew = sumDew + value['Dew']
    sumRain = sumRain + value['Rain']
    cnt = cnt + 1
    if today != preday:
        print(preday)
        # print(minTem, maxTem, sumTem / cnt, sumPre / cnt, sumPre1 / cnt, sumHum / cnt, sumWin / cnt,
        #       sumVis / cnt, sumDew / cnt, sumRain / cnt)
        df1.loc[len(df1)] = [preday, minTem, maxTem, sumTem / cnt, sumPre / cnt, sumPre1 / cnt, minHum, maxHum,
                             sumHum / cnt, minWin, maxWin, sumWin / cnt, sumVis / cnt, sumDew / cnt, sumRain * 4 / cnt]
        cnt = 0
        minTem, maxTem = 99999.0, -99999.0
        minHum, maxHum = 99999.0, -99999.0
        minWin, maxWin = 99999.0, -99999.0
        sumTem, sumPre, sumPre1, sumHum, sumWin, sumVis, sumDew, sumRain = 0, 0, 0, 0, 0, 0, 0, 0
    # print(today, preday)
    preday = today

print(df1)
# for i in range(2005, 2024):
#     for j in range(1, len(months) + 1):
#         x = months[j - 1] + 1
#         if checkRun(i) and j == 2:
#             x = x + 1
#         for k in range(1, x):
#             if yy == i and mm == j and dd == k:

# print(yy, mm, dd)



365
2005/2/1
2005/2/2
2005/2/3
2005/2/4
2005/2/5
2005/2/6
2005/2/7
2005/2/8
2005/2/9
2005/2/10
2005/2/11
2005/2/12
2005/2/13
2005/2/14
2005/2/15
2005/2/16
2005/2/17
2005/2/18
2005/2/19
2005/2/20
2005/2/21
2005/2/22
2005/2/23
2005/2/24
2005/2/25
2005/2/26
2005/2/27
2005/2/28
2005/3/1
2005/3/2
2005/3/3
2005/3/4
2005/3/5
2005/3/6
2005/3/7
2005/3/8
2005/3/9
2005/3/10
2005/3/11
2005/3/12
2005/3/13
2005/3/14
2005/3/15
2005/3/16
2005/3/17
2005/3/18
2005/3/19
2005/3/20
2005/3/21
2005/3/22
2005/3/23
2005/3/24
2005/3/25
2005/3/26
2005/3/27
2005/3/28
2005/3/29
2005/3/30
2005/3/31
2005/4/1
2005/4/2
2005/4/3
2005/4/4
2005/4/5
2005/4/6
2005/4/7
2005/4/8
2005/4/9
2005/4/10
2005/4/11
2005/4/12
2005/4/13
2005/4/14
2005/4/15
2005/4/16
2005/4/17
2005/4/18
2005/4/19
2005/4/20
2005/4/21
2005/4/22
2005/4/23
2005/4/24
2005/4/25
2005/4/26
2005/4/27
2005/4/28
2005/4/29
2005/4/30
2005/5/1
2005/5/2
2005/5/3
2005/5/4
2005/5/5
2005/5/6
2005/5/7
2005/5/8
2005/5/9
2005/5/10
2005/5/11
2005/5/12
2005/5/13
2005/5/14
20

In [8]:
df1.to_csv(f'data/groupByCZ/JiChang_2005_2.csv')

In [2]:
import math


def haversine(lat1, lon1, lat2, lon2, r=6371):
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Calculate haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))

    # Return distance in kilometers
    return r * c

In [3]:
haversine(39.9, 116.4, 31.2, 121.5)

1071.2857795943337

In [13]:
loc = pd.read_csv('data/苕溪等五溪.csv')
Lat = loc['CLTTD'].tolist()
Lon = loc['CLGTD'].tolist()

dis_to_cent = []
for i in range(len(Lat) - 1):
    dis_to_cent.append(haversine(Lat[i], Lon[i], Lat[5], Lon[5]))

print(dis_to_cent)
dis_to_cent = [round(i, 2) for i in dis_to_cent]
dis_to_cent = [1 - i / 259.19 for i in dis_to_cent]

print(dis_to_cent)

[31.55608604850908, 22.639414351627384, 19.724205775114342, 19.400416860488672, 19.269159378609352, 0.0, 135.53621643867083, 153.56974377726536, 138.23315956581482, 259.190207545108]
[0.8782360430572167, 0.9126509510397778, 0.9239168177784637, 0.9251514333114703, 0.9256529958717543, 1.0, 0.47706315830086043, 0.4075002893630155, 0.46668467147652304, 0.0]


In [7]:

# rawdata1 = rawdata1.loc[:, ~rawdata1.columns.str.contains('^Unnamed')]
# rawdata1 = rawdata1.drop(columns='_c0')
# rawdata1 = rawdata1.drop(columns='_c2')
# rawdata1 = rawdata1.drop(columns='_c4')
# rawdata1 = rawdata1.drop(columns='_c5')
# rawdata1 = rawdata1.drop(columns='_c6')
# rawdata1 = rawdata1.drop(columns='_c7')
# # rawdata2 = rawdata2.drop(columns='RCD')
# rawdata1.columns = ['TM', 'DRY']
# print(rawdata1)
# # print(rawdata2)
# rawdata1.to_csv('data/groupByCZ/ronghe.csv', index=False)
# # rawdata3 = pd.read_csv('data/st_pptn_day.csv')
# #
# # data1 = rawdata1[:200]
# # data2 = rawdata3[:200]
# # print(data1, data2)
# # plt.figure(figsize=(16, 9))
# # plt.plot(rawdata3['DRP'])
# # # plt.plot(rawdata1['DRY'])
# # # plt.xticks(range(0, rawdata1.shape[0], 365), rawdata1['TM'].loc[::365], rotation=90)
# # # plt.xticks(range(0, rawdata1.shape[0], 365), rawdata1['_c1'].loc[::365], rotation=90)
# # plt.tight_layout()
# # # print(rawdata1)
# # # print(rawdata2)

KeyError: "['_c4'] not found in axis"

SELECT * FROM `st_stbprp_b` WHERE RVNM = '苕溪'
SELECT * FROM `st_stbprp_b` WHERE STCD = '63023402'
SELECT COUNT(*) FROM `st_dayev_r` WHERE STCD = '63002400'
SELECT COUNT(*) FROM `st_rvav_r` WHERE STCD = '63002400'
SELECT COUNT(*) FROM `st_soil_r` WHERE STCD = '63023402'
SELECT COUNT(*) FROM `st_rsvrav_r` WHERE STCD = '70112000'
SELECT COUNT(*) FROM `qymyl_ls` WHERE RCD = '330502'
41FHBA1AA0000R1
41FHBA1B00000M1
41FHBA1B2B000R1
41FHBA1BA0000R1
41FHBA1BB0000R1
41FHBA1BC0000L1
41GABAD000000R1
41GAF1A000000L1
41GAFDA000000R1
41GB1CA000000R1

In [43]:
sppd = spark.read.csv('data/QYMYL_LS1.csv')
# df1 = sppd.select('_c0', '_c1').groupby(sppd['_c0']).count()
# df1 = df1.sort(df1['count'].desc(), df1['_c0'])
L = ['41FHBA1AA0000R1',
     '41FHBA1B00000M1',
     '41FHBA1B2B000R1',
     '41FHBA1BA0000R1',
     '41FHBA1BB0000R1',
     '41FHBA1BC0000L1',
     '41GABAD000000R1',
     '41GAF1A000000L1',
     '41GAFDA000000R1',
     '41GB1CA000000R1']

df2 = sppd.filter(sppd['_c1'] == '41FHBA1AA0000R1')
rawdata = df2.toPandas()
df1 = pd.DataFrame()
df1.insert(loc=0, column='date', value=rawdata._c2.tolist())
for i in range(len(L) - 1):
    df2 = sppd.filter(sppd['_c1'] == L[i])
    rawdata = df2.toPandas()
    rawdata._c4.astype('float')
    myl = rawdata['_c4'].tolist()
    myl = np.array(list(map(float, myl)))
    myl = myl * dis_to_cent[i]
    # rawdata1 = rawdata
    # rawdata1 = rawdata1.drop(columns='_c0')
    # rawdata1 = rawdata1.drop(columns='_c1')
    # rawdata1 = rawdata1.drop(columns='_c3')
    # rawdata1 = rawdata1.drop(columns='_c5')
    # rawdata1 = rawdata1.drop(columns='_c6')
    # rawdata1.columns = ['TM', 'DYE']
    # rawdata1.to_csv(f'data/groupByCZ/{L[i]}MYL.csv')
    df1.insert(loc=i+1, column=f'MYL{i+1}', value=myl)
    # print(myl)

print(df1)

                      date      MYL1      MYL2      MYL3      MYL4      MYL5  \
0      1954-01-01 00:00:00  4.391180  3.833134  4.434801  4.440727  4.350569   
1      1954-01-02 00:00:00  0.000000  0.273795  0.461958  0.462576  0.462826   
2      1954-01-03 00:00:00  0.000000  0.365060  0.646742  0.555091  0.647957   
3      1954-01-04 00:00:00  0.790412  1.095181  1.293484  0.925151  1.481045   
4      1954-01-05 00:00:00  0.000000  0.000000  0.000000  0.000000  0.000000   
...                    ...       ...       ...       ...       ...       ...   
24467  2020-12-27 00:00:00  0.000000  0.000000  0.000000  0.000000  0.000000   
24468  2020-12-28 00:00:00  0.000000  0.000000  0.000000  0.000000  0.000000   
24469  2020-12-29 00:00:00  5.093769  5.749701  7.760901  6.383545  7.868050   
24470  2020-12-30 00:00:00  0.000000  0.000000  0.000000  0.000000  0.000000   
24471  2020-12-31 00:00:00  0.000000  0.000000  0.000000  0.000000  0.000000   

       MYL6      MYL7      MYL8      MY

In [44]:
df1.to_csv('data/groupByCZ/330112MYL.csv')

In [66]:
target = sppd.filter(sppd['_c0'] == '63003085')
target = target.toPandas()
target._c2 = target._c2.astype(float)
print(target)
# target.to_csv('data/groupByCZ/62414660st_pptn_day.csv', index=False)

           _c0                  _c1   _c2                  _c3
0     63003085  2017-06-02 00:00:00   0.0  2020-04-22 14:46:11
1     63003085  2017-06-03 00:00:00   0.0  2020-04-22 14:49:38
2     63003085  2017-06-04 00:00:00   0.0  2020-04-22 14:49:58
3     63003085  2017-06-05 00:00:00   0.0  2020-04-22 14:50:19
4     63003085  2017-06-06 00:00:00  28.0  2020-04-22 14:50:39
...        ...                  ...   ...                  ...
1096  63003085  2022-10-19 00:00:00   0.0  2022-10-20 00:00:06
1097  63003085  2022-10-20 00:00:00   0.0  2022-10-21 00:00:03
1098  63003085  2022-10-21 00:00:00   0.0  2022-10-22 00:01:41
1099  63003085  2022-10-22 00:00:00   0.0  2022-10-23 00:05:32
1100  63003085  2022-10-23 00:00:00   0.0  2022-10-24 00:14:39

[1101 rows x 4 columns]


In [41]:
# rawdata2 = pd.read_csv(f'data/groupByCZ/330000MYL1.csv')
QYMYL_LS = pd.read_csv(f'data/groupByCZ/330502QYMYL_LS.csv')
st_rvav_r = pd.read_csv(f'data/groupByCZ/63202300st_rvav_r.csv')
st_dayev_r = pd.read_csv(f'data/groupByCZ/63000600st_dayev_r.csv')
st_rsvrav_r = pd.read_csv(f'data/groupByCZ/70112000st_rsvrav_r.csv')
for k, v in QYMYL_LS.iterrows():
    if v['TM'] < '2015-10-01' or v['TM'] > '2020-12-31':
        QYMYL_LS = QYMYL_LS.drop(k)
for k, v in st_rvav_r.iterrows():
    if v['TM'] < '2015-10-01' or v['TM'] > '2020-12-31':
        st_rvav_r = st_rvav_r.drop(k)
for k, v in st_dayev_r.iterrows():
    if v['TM'] < '2015-10-01' or v['TM'] > '2020-12-31':
        st_dayev_r = st_dayev_r.drop(k)
for k, v in st_rsvrav_r.iterrows():
    if v['TM'] < '2015-10-01' or v['TM'] > '2020-12-31':
        st_rsvrav_r = st_rsvrav_r.drop(k)
L = [QYMYL_LS, st_rvav_r, st_dayev_r, st_rsvrav_r]
for i in L:
    print(i)
# plt.figure(figsize=(16, 9))
# plt.plot(rawdata1._c2)
# plt.plot(rawdata['DRY'])
# plt.xticks(range(0, target.shape[0], 10), target['_c1'].loc[::10], rotation=90)
# plt.savefig('data/img/160_90.jpg')
# df1 = pd.DataFrame(columns=['TM', 'DRY', 'DRP'])
# print(df1)
# plt.show()
# rawdata1.count

       Unnamed: 0                   TM    DRP
22553       22553  2015-10-01 00:00:00   2.36
22554       22554  2015-10-02 00:00:00   0.15
22555       22555  2015-10-03 00:00:00   0.00
22556       22556  2015-10-04 00:00:00  48.70
22557       22557  2015-10-05 00:00:00   3.38
...           ...                  ...    ...
24466       24466  2020-12-26 00:00:00   0.07
24467       24467  2020-12-27 00:00:00   0.01
24468       24468  2020-12-28 00:00:00   0.29
24469       24469  2020-12-29 00:00:00   7.69
24470       24470  2020-12-30 00:00:00   0.09

[1918 rows x 3 columns]
      Unnamed: 0                   TM    AVQ
177          177  2015-10-01 08:00:00  65.60
178          178  2015-10-02 08:00:00  57.40
179          179  2015-10-03 08:00:00  31.10
180          180  2015-10-04 08:00:00  21.80
181          181  2015-10-05 08:00:00  14.90
...          ...                  ...    ...
1961        1961  2020-12-25 08:00:00   0.00
1962        1962  2020-12-26 08:00:00   4.80
1963        1963  

In [43]:
df1 = pd.DataFrame(columns=['TM', 'AVINQ', 'AVOTQ', 'DRP', 'DYE', 'AVQ'])
for k1, v1 in st_rsvrav_r.iterrows():
    s = v1['TM'][:10]
    # print(s)
    flag1, flag2, flag3 = 0, 0, 0
    v3, v4, v5, v6 = 0, 0, 0, 0
    for k2, v2 in QYMYL_LS.iterrows():
        if s == v2['TM'][:10]:
            v3 = v2['DRP']
            break
    for k2, v2 in st_dayev_r.iterrows():
        if s == v2['TM'][:10]:
            v4 = v2['DYE']
            break
    for k2, v2 in st_rvav_r.iterrows():
        if s == v2['TM'][:10]:
            # v5 = v2['AVZ']
            v6 = v2['AVQ']
            break
    df1.loc[len(df1)] = [s, v1['AVINQ'], v1['AVOTQ'], v3, v4, v6]
df1.fillna(method='ffill', inplace=True)
df1.fillna(0, inplace=True)
print(df1)

              TM  AVINQ  AVOTQ   DRP  DYE    AVQ
0     2015-10-01  270.0  237.0  2.36  0.0  65.60
1     2015-10-01  140.0  374.0  2.36  0.0  65.60
2     2015-10-01  153.0  324.0  2.36  0.0  65.60
3     2015-10-02  170.0   15.4  0.15  3.0  57.40
4     2015-10-03  129.0   35.3  0.00  3.6  31.10
...          ...    ...    ...   ...  ...    ...
2154  2020-12-26   24.4  481.0  0.07  0.0   4.80
2155  2020-12-27  131.0  489.0  0.01  0.0   2.43
2156  2020-12-28   72.0  490.0  0.29  0.0   0.00
2157  2020-12-29   77.1  490.0  7.69  0.7   2.50
2158  2020-12-30  141.0  488.0  0.09  0.6   3.01

[2159 rows x 6 columns]


In [44]:
df1.to_csv('data/groupByCZ/2015_2020.csv')

In [25]:
st_rvav_r = spark.read.csv('data/st_rvav_r.csv')
spdf = st_rvav_r.select(st_rvav_r['_c0']).groupby(st_rvav_r['_c0']).count()
spdf = spdf.sort(spdf['count'].desc())
spdf.show()

+--------+-----+
|     _c0|count|
+--------+-----+
|63002400| 2340|
|63202300| 2103|
|63010800| 1712|
|63201751| 1590|
|63303100| 1509|
|70100500|  855|
|63101301|  543|
|63102150|  523|
|70112001|  254|
|70600600|  193|
|63001200|  143|
|70200800|  138|
|70117700|  134|
|70100900|  102|
|63007400|  101|
|70508800|   94|
|70108400|   94|
|70115250|   88|
|70610200|   87|
|70400800|   85|
+--------+-----+
only showing top 20 rows



In [3]:
df = pd.read_csv('data/groupByCZ/2013_2022DRPDRY.csv')
df.fillna(method='ffill', inplace=True)
df.fillna(0, inplace=True)
df.to_csv('data/groupByCZ/2013_2022DRPDRY.csv')
print(df)

      Unnamed: 0          TM   DRP  DRY
0              0  2013-01-12  8.70  0.3
1              1  2013-01-13  1.11  0.3
2              2  2013-01-14  0.05  0.3
3              3  2013-01-15  0.05  0.2
4              4  2013-01-16  0.07  0.4
...          ...         ...   ...  ...
2905        2905  2020-12-26  0.35  0.0
2906        2906  2020-12-27  0.10  0.0
2907        2907  2020-12-28  0.01  0.0
2908        2908  2020-12-29  4.81  0.7
2909        2909  2020-12-30  0.11  0.6

[2910 rows x 4 columns]


In [ ]:
file = '330112MYL'
rawdata = pd.read_csv(f'data/groupByCZ/{file}.csv')
# rawdata.columns = ['', 'RCD', 'TM', 'INTV', 'DRP', 'MODITIME', 'SOURCE']
rawdata = rawdata.loc[:, ~rawdata.columns.str.contains('^Unnamed')]
plt.figure(figsize=(20, 10))
# plt.plot(dataset['avgRain'], label='avgRain')
# for i in range(1, 9):
#     plt.plot(rawdata[f'MYL{i}'], label=f'MYL{i}')
plt.plot(rawdata[f'MYL1'], label=f'MYL1')
plt.xticks(range(0, rawdata.shape[0], 365), rawdata['date'].loc[::365], rotation=90)
# plt.tight_layout()
plt.legend()
plt.show()
# dataset.DRP = ema_corr(dataset.DRP, 0.5)
# dataset.DRY = ema_corr(dataset.DRY, 0.1)
# print(dataset)
# plt.plot(dataset.AVZ)
# plt.plot(dataset.DRY)
# plt.show()

# corrmatrix = rawdata.corr(method='pearson')
# print(corrmatrix.loc['avgRain'])

NameError: name 'np' is not defined